# 2D Filter demo

## Requirements

Custom video overlay for Pynq-Z1 v2.0 image
HDMI In 1280x720 (720p) source connected to the board

This demo allows the pushbuttons on the board to be used to select different filters. 
## Slideshow

* If the resolution is incorrectly set the first time the board is connected, set the resolution, and rerun all the cells in this notebook
 
This demo is intended to be run as a slideshow. 

Go to View > Cell Toolbar > Slideshow to see the *Slideshow options* for each cell. From here you can select which slides will be included or excluded from the presentation. 

Code cells can be executed from the slideshow view by clicking the code cell, and pressing CTRL + ENTER


## Instructions to run the demo:

* Before entering the slideshow view, click Cell > Run All to execute all the code in the notebook. 

In slideshow mode, use the cursor keys (left right) to navigate through the presentation

* Press Alt + r to enter the slideshow and hide this view. (Exit slideshow mode with the same keys Alt + r)

In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("./filter.bit")

In [2]:
from pynq.lib.video import *
rgba = PixelFormat(32, COLOR_IN_RGB, COLOR_OUT_RGB)
mode = VideoMode(1280, 720, 32)

base.video.hdmi_in.configure(rgba)

base.video.hdmi_in.start()

base.video.hdmi_out.configure(mode, rgba)
base.video.hdmi_out.start()

In [3]:
base.video.hdmi_in.mode # Check input mode; Should be 1280x 720

VideoMode: width=1280 height=720 bpp=32

This should turn on the monitor but nothing will be displayed yet.

In [4]:
filter2d = base.video.filter2d_0

coeffs = np.frombuffer(filter2d.mmio.mem, np.int16, 96, 64)

The arguments to `np.frombuffer` are the buffer object, the type we want, the number of elements and the offset in bytes. 96 is chosen because that's the number of 16-bit integers that can fit into the range 0x40 to 0xFF and 64 is the offset of the coeffs1 array.

Now we have the master array we can get the slices for each particular set of coefficients.

In [5]:
c1 = coeffs[0:25].reshape((5,5))
c2 = coeffs[32:57].reshape((5,5))
c3 = coeffs[64:89].reshape((5,5))

Note that while we have specified the types here as a 16-bit integer the type used by the hardware is a 16-bit fixed-point number so we need multiply the value of the coefficients by 256 when writing them to the hardware. Finally we can make our filter pass through the signal intact and (hopefully) see something on the screen.

Make sure something is being displayed on HDMI out

In [6]:
base.video.hdmi_in.tie(base.video.hdmi_out)

In [ ]:
# Some options for filters: https://en.wikipedia.org/wiki/Kernel_%28image_processing%29

default_mask = np.frombuffer(filter2d.mmio.mem, np.int16, 96, 64)
default_mask = [[0, 0,  0, 0, 0],
                [0, 0,  0, 0, 0],
                [0, 0,256, 0, 0],
                [0, 0,  0, 0, 0],
                [0, 0,  0, 0, 0]]

blank_mask = np.frombuffer(filter2d.mmio.mem, np.int16, 96, 64)
blank_mask = [[0, 0,  0, 0, 0],
                [0, 0,  0, 0, 0],
                [0, 0, 0, 0, 0],
                [0, 0,  0, 0, 0],
                [0, 0,  0, 0, 0]]

sobel_mask = np.frombuffer(filter2d.mmio.mem, np.int16, 96, 64)
sobel_mask = [[0,0,  1,0,0],
              [0,1,  2,1,0],
              [1,2,-16,2,1],
              [0,1,  2,1,0],
              [0,0,  1,0,0]]
sobel_mask = 256*np.array(sobel_mask)

enhance_mask = np.frombuffer(filter2d.mmio.mem, np.int16, 96, 64)
enhance_mask = [[0, 0,  5, 0, 0],
                [0, 5,  9, 5, 0],
                [5, 9,256, 9, 5],
                [0, 5,  9, 5, 0],
                [0, 0,  5, 0, 0]]

sharpen_mask = np.frombuffer(filter2d.mmio.mem, np.int16, 96, 64)
sharpen_mask = [[0, 0,  0, 0, 0],
                [0, 0, -1, 0, 0],
                [0,-1,  5,-1, 0],
                [0, 0, -1, 0, 0],
                [0, 0,  0, 0, 0]]
sharpen_mask = 256*np.array(sharpen_mask)

gaussian_blur_mask = np.frombuffer(filter2d.mmio.mem, np.int16, 96, 64)
gaussian_blur_mask = [[1, 4,  6, 4, 1],
                      [4,16, 24,16, 4],
                      [6,24, 36,24, 6],
                      [4,16, 24,16, 4],
                      [1, 4,  6, 4, 1]]

c1[:] = default_mask[:]
c2[:] = default_mask[:]
c3[:] = default_mask[:]

# Video filter

Change the mask in the cell below to change the function for each button.

1. Selects Default (pass through)
2. Selects edge detection
3. Selects a blur filter
4. Cycles through Red/Blue/Green channels 

In [ ]:
import time
count = 0
while (base.switches[0].read()==0):
    if (base.buttons[0].read()==1):
        c1[:] = default_mask[:]
        c2[:] = default_mask[:]
        c3[:] = default_mask[:]
        count = 0
        time.sleep(.1)
    elif (base.buttons[1].read()==1):
        c1[:] = sobel_mask[:]
        c2[:] = sobel_mask[:]
        c3[:] = sobel_mask[:]
        count = 0
        time.sleep(.1)
    elif (base.buttons[2].read()==1):
        c1[:] = gaussian_blur_mask[:]
        c2[:] = gaussian_blur_mask[:]
        c3[:] = gaussian_blur_mask[:]
        count = 0
        time.sleep(.1)
    elif (base.buttons[3].read()==1):
        if count is 0:
            c1[:] = default_mask[:]
            c2[:] = blank_mask[:]
            c3[:] = blank_mask[:]
            count += 1
        elif count is 1:
            c1[:] = blank_mask[:]
            c2[:] = blank_mask[:]
            c3[:] = default_mask[:]
            count += 1
        elif count is 2:
            c1[:] = blank_mask[:]
            c2[:] = default_mask[:]
            c3[:] = blank_mask[:]
            count = 0
        time.sleep(0.5)